<a href="https://colab.research.google.com/github/viniciusriosfuck/python-financas/blob/main/Python_para_finan%C3%A7as_CAPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python para finanças - CAPM (Capital Asset Pricing Model)

## Importação das bibliotecas e tratamento da base de dados

$$ R_i = R_f + \beta_i\cdot(R_m-R_f)  $$
* $R_f$: investimento sem risco (CDB)
* $R_m$: retorno esperado do mercado (IBOVESPA)
* $\beta$: comparação entre carteira e mercado (IBOVESPA) ~ prêmio
    * $=1$ forte correlação com mercado
    * $<1$ defensivo, menos volátil
    * $>1$ agressivo, menos volátil
* Leitura CAPM: retorno dado risco por investir na ação


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

import pandas as pd
from pandas_datareader import data

acoes = ['GOLL4.SA', 'CVCB3.SA', 'WEGE3.SA', 'MGLU3.SA', 'TOTS3.SA', 'BOVA11.SA']
acoes_df = pd.DataFrame()
for acao in acoes:
  acoes_df[acao] = data.DataReader(acao, data_source='yahoo', start='2015-01-01')['Close']

acoes_df = acoes_df.rename(columns={'GOLL4.SA': 'GOL', 'CVCB3.SA': 'CVC', 'WEGE3.SA': 'WEGE',
                                    'MGLU3.SA': 'MGLU', 'TOTS3.SA': 'TOTS', 'BOVA11.SA': 'BOVA'})

acoes_df.dropna(inplace=True)
acoes_df.to_csv('acoes.csv')

dataset_raw = pd.read_csv('acoes.csv')

days_bolsa = 246

In [ ]:
dataset = pd.read_csv('acoes.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset

,Date,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,2015-01-02,14.990000,15.200000,11.846153,0.232812,11.910702,47.259998
1,2015-01-05,14.850000,15.000000,11.926923,0.237187,11.544731,46.320000
2,2015-01-06,15.210000,14.800000,11.750000,0.234062,10.822770,46.580002
3,2015-01-07,14.550000,14.670000,11.615384,0.241875,10.746248,48.150002
4,2015-01-08,14.270000,14.150000,11.811538,0.240000,10.995774,48.509998
...,...,...,...,...,...,...,...
1498,2021-01-22,22.670000,18.370001,88.260002,26.000000,30.150000,113.180000
1499,2021-01-26,22.209999,17.440001,88.949997,25.750000,30.110001,112.970001
1500,2021-01-27,23.030001,18.020000,88.690002,25.790001,28.740000,112.970001
1501,2021-01-28,24.610001,19.240000,87.970001,26.190001,29.459999,113.910004


In [ ]:
dataset.drop(columns=['Date'], inplace = True)

In [ ]:
dataset_normalizado = dataset.copy()
dataset_normalizado /= dataset_normalizado.iloc[0]
# for i in dataset.columns:
#   dataset_normalizado[i] = dataset[i] / dataset[i][0]
dataset_normalizado

,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.990660,0.986842,1.006818,1.018792,0.969274,0.980110
2,1.014676,0.973684,0.991883,1.005369,0.908659,0.985612
3,0.970647,0.965132,0.980519,1.038928,0.902235,1.018832
4,0.951968,0.930921,0.997078,1.030875,0.923184,1.026449
...,...,...,...,...,...,...
1498,1.512342,1.208553,7.450520,111.678091,2.531337,2.394837
1499,1.481654,1.147368,7.508766,110.604263,2.527979,2.390394
1500,1.536358,1.185526,7.486819,110.776080,2.412956,2.390394
1501,1.641761,1.265789,7.426039,112.494203,2.473406,2.410284


In [ ]:
dataset_taxa_retorno = (dataset_normalizado / dataset_normalizado.shift(1)) - 1
dataset_taxa_retorno

,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.009340,-0.013158,0.006818,0.018792,-0.030726,-0.019890
2,0.024242,-0.013333,-0.014834,-0.013175,-0.062536,0.005613
3,-0.043392,-0.008784,-0.011457,0.033380,-0.007070,0.033705
4,-0.019244,-0.035447,0.016887,-0.007752,0.023220,0.007477
...,...,...,...,...,...,...
1498,-0.003954,-0.046210,0.006960,0.020008,0.013445,-0.011183
1499,-0.020291,-0.050626,0.007818,-0.009615,-0.001327,-0.001855
1500,0.036920,0.033257,-0.002923,0.001553,-0.045500,0.000000
1501,0.068606,0.067703,-0.008118,0.015510,0.025052,0.008321


In [ ]:
dataset_taxa_retorno.fillna(0, inplace=True)
dataset_taxa_retorno.head()

,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.009340,-0.013158,0.006818,0.018792,-0.030726,-0.019890
2,0.024242,-0.013333,-0.014834,-0.013175,-0.062536,0.005613
3,-0.043392,-0.008784,-0.011457,0.033380,-0.007070,0.033705
4,-0.019244,-0.035447,0.016887,-0.007752,0.023220,0.007477


In [ ]:
dataset_taxa_retorno.mean() * days_bolsa

GOL     0.393045
CVC     0.190536
WEGE    0.386380
MGLU    0.977685
TOTS    0.216484
BOVA    0.175631
dtype: float64

## Cálculo do parâmetro BETA para um ativo

### BETA com regressão linear

- Introdução a regressão linear: https://www.youtube.com/watch?v=ltRVgNsZBXE

In [ ]:
figura = px.scatter(dataset_taxa_retorno, x='BOVA', y='MGLU', title='BOVA x MGLU')
figura.show()

In [ ]:
beta, alpha = np.polyfit(x=dataset_taxa_retorno['BOVA'], y=dataset_taxa_retorno['MGLU'], deg=1)
print('beta:', beta, 'alpha:', alpha, 'alpha (%):', alpha * 100)

beta: 1.09413544021467 alpha: 0.0031931733140793178 alpha (%): 0.3193173314079318


In [ ]:
1.118179637527429 * 0.07041056 + 0.0034252907778759356

0.08215694523677923

In [ ]:
figura = px.scatter(dataset_taxa_retorno, x='BOVA', y='MGLU', title='BOVA x MGLU')
figura.add_scatter(x=dataset_taxa_retorno['BOVA'], y=beta*dataset_taxa_retorno['BOVA']+alpha)
figura.show()

### BETA com covariância e variância

In [ ]:
# matriz_covariancia = dataset_taxa_retorno.drop(columns=['GOL', 'CVC', 'WEGE', 'TOTS']).cov() * 246
matriz_covariancia = dataset_taxa_retorno[['MGLU', 'BOVA']].cov() * days_bolsa
matriz_covariancia

,MGLU,BOVA
MGLU,0.427176,0.079154
BOVA,0.079154,0.072344


In [ ]:
cov_mglu_bova = matriz_covariancia.iloc[1, 0]
cov_mglu_bova

0.07915448591072692

In [ ]:
variancia_bova = dataset_taxa_retorno['BOVA'].var()*days_bolsa
variancia_bova

0.07234432137139883

In [ ]:
beta_mglu = cov_mglu_bova / variancia_bova
beta_mglu

1.0941354402146686

## Cálculo CAPM para uma ação

In [ ]:
beta

1.09413544021467

In [ ]:
rm = dataset_taxa_retorno['BOVA'].mean()*days_bolsa
rm

0.17563112842946949

In [ ]:
taxa_selic_historico = np.array([12.75, 14.25, 12.25, 6.5, 5.0, 2.0])
rf = taxa_selic_historico.mean() / 100
rf

0.08791666666666666

In [ ]:
capm_mglu = rf + (beta * (rm - rf))
capm_mglu

0.18388816790070378

## Cálculo do BETA para todas as ações

In [ ]:
betas = []
alphas = []
for ativo in dataset_taxa_retorno.columns[0:-1]:
  #print(ativo)
  beta, alpha = np.polyfit(dataset_taxa_retorno['BOVA'], dataset_taxa_retorno[ativo], 1)
  betas.append(beta)
  alphas.append(alpha)

In [ ]:
betas

[1.7617471576427457,
 1.1277105902464735,
 0.6922645523345142,
 1.09413544021467,
 0.6864840534562242]

In [ ]:
alphas

[0.00033994877173849935,
 -3.0588816431501024e-05,
 0.0010764110353505781,
 0.0031931733140793178,
 0.00038990073660126126]

In [ ]:
def visualiza_betas_alphas(betas, alphas):
    # for i, ativo in enumerate(dataset_taxa_retorno.columns[0:-1]):
    # #print(i, ativo)
    #     print(ativo, 'beta:', betas[i], 'alpha:', alphas[i] * 100)
#   return dict(zip(dataset_taxa_retorno.columns[0:-1], betas))
    return pd.DataFrame(
        columns=["ativo","beta","alpha"],
        data = list(zip(
            dataset_taxa_retorno.columns[0:-1],
            betas,
            [x*100 for x in alphas]
            ))
        )




alphas: $>0$ alpha melhor que BOVA

In [ ]:
visualiza_betas_alphas(betas, alphas)

,ativo,beta,alpha
0,GOL,1.761747,0.033995
1,CVC,1.127711,-0.003059
2,WEGE,0.692265,0.107641
3,MGLU,1.094135,0.319317
4,TOTS,0.686484,0.038990


In [ ]:
np.array(alphas).mean() * 100

0.09937690082676312

## Cálculo CAPM para o portfólio

In [ ]:
rf

0.08791666666666666

In [ ]:
rm

0.17563112842946949

In [ ]:
capm_empresas = []
for i, ativo in enumerate(dataset_taxa_retorno.columns[0:-1]):
  #print(i, ativo)
  capm_empresas.append(rf + (betas[i] * (rm - rf)))

In [ ]:
capm_empresas

[0.24244737036144787,
 0.18683319411434876,
 0.14863827927215623,
 0.18388816790070378,
 0.14813124592432653]

In [ ]:
def visualiza_capm(capm):
#   for i, ativo in enumerate(dataset_taxa_retorno.columns[0:-1]):
#     print(ativo, 'CAPM:', capm[i] * 100)
  return pd.DataFrame(
        columns=["ativo","CAPM"],
        data = list(zip(
            dataset_taxa_retorno.columns[0:-1],
            [x*100 for x in capm]
            ))
        )

In [ ]:
visualiza_capm(capm_empresas)

,ativo,CAPM
0,GOL,24.244737
1,CVC,18.683319
2,WEGE,14.863828
3,MGLU,18.388817
4,TOTS,14.813125


In [ ]:
pesos = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

In [ ]:
capm_portfolio = np.sum(capm_empresas * pesos) * 100
capm_portfolio

18.198765151459664

## Exercício

In [ ]:
acoes_ex = ['ABEV3.SA', 'ODPV3.SA', 'VIVT3.SA', 'PETR3.SA', 'BBAS3.SA', 'BOVA11.SA']
acoes_ex_df = pd.DataFrame()
for acao in acoes_ex:
  acoes_ex_df[acao] = data.DataReader(acao, data_source='yahoo', start='2015-01-01')['Close']
acoes_ex_df.dropna(inplace=True)
acoes_ex_df = acoes_ex_df.rename(columns={
    'ABEV3.SA': 'AMBEV', 'ODPV3.SA': 'ODONTOPREV', 'VIVT3.SA': 'VIVO',
    'PETR3.SA': 'PETROBRAS', 'BBAS3.SA': 'BBRASIL', 'BOVA11.SA': 'BOVA'})
acoes_ex_df.to_csv('acoes_ex.csv')

In [ ]:
dataset = pd.read_csv('acoes_ex.csv')
dataset.drop(columns = ['Date'], inplace=True)
dataset_normalizado = dataset.copy()

# for i in dataset.columns:
#   dataset_normalizado[i] = dataset[i] / dataset[i][0]
dataset_normalizado /= dataset_normalizado.iloc[0]

dataset_taxa_retorno = (dataset_normalizado / dataset_normalizado.shift(1)) - 1
dataset_taxa_retorno.fillna(0, inplace=True)
dataset_taxa_retorno

,AMBEV,ODONTOPREV,VIVO,PETROBRAS,BBRASIL,BOVA
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.018102,0.026681,-0.019831,-0.081111,-0.020751,-0.019890
2,0.038144,-0.007276,-0.024818,-0.025393,0.013977,0.005613
3,0.014084,0.014660,0.034301,0.048387,0.044020,0.033705
4,-0.001208,-0.003096,0.040653,0.067456,0.003407,0.007477
...,...,...,...,...,...,...
1498,-0.037156,-0.012474,0.005705,-0.013884,-0.022235,-0.011183
1499,0.024617,0.007018,0.017245,-0.003610,-0.018851,-0.001855
1500,0.000000,-0.011847,0.018514,0.013768,0.029277,0.000000
1501,0.009091,0.024683,0.001095,0.022159,0.023407,0.008321


In [ ]:
betas = []
alphas = []
for ativo in dataset_taxa_retorno.columns[0:-1]:
  beta, alpha = np.polyfit(dataset_taxa_retorno['BOVA'], dataset_taxa_retorno[ativo], 1)
  betas.append(beta)
  alphas.append(alpha)

In [ ]:
# O retorno do BBRASIL é 37% mais volátil que o mercado
# O retorno da PETROBRAS excedeu o mercado em 0.01%
# O retorno da AMBEV ficou abaixo do mercado em 0.03%
visualiza_betas_alphas(betas, alphas)

,ativo,beta,alpha
0,AMBEV,0.600502,-0.030364
1,ODONTOPREV,0.453899,0.015607
2,VIVO,0.522589,-0.004572
3,PETROBRAS,1.513626,0.024136
4,BBRASIL,1.371964,-0.026370


In [ ]:
np.array(alphas).mean() * 100

-0.004312507687018539

In [ ]:
rm = dataset_taxa_retorno['BOVA'].mean() * days_bolsa
taxa_selic_historico = np.array([12.75, 14.25, 12.25, 6.5, 5.0, 2.0]).mean() / 100
rm, rf

(0.17563112842946949, 0.08791666666666666)

In [ ]:
capm_empresas = []
for i, ativo in enumerate(dataset_taxa_retorno.columns[0:-1]):
  capm_empresas.append(rf + (betas[i] * (rm - rf)))

In [ ]:
# Se investirmos na AMBEV, ganharemos 12.5% de retorno para ser compensado pelo risco que corremos
visualiza_capm(capm_empresas)

,ativo,CAPM
0,AMBEV,14.058937
1,ODONTOPREV,12.773017
2,VIVO,13.375531
3,PETROBRAS,22.068354
4,BBRASIL,20.825775


In [ ]:
pesos = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

In [ ]:
# Se investirmos no portfólio, ganharemos 14.4% de retorno para ser compensado pelo risco que corremos
capm_portfólio = np.sum(capm_empresas * pesos) * 100
capm_portfólio

16.620322598054763